In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.decomposition import TruncatedSVD

In [2]:
#read train file
train_data = open('C:/Users/saish/Desktop/A_PR2/train.dat','r')
tr_lines = train_data.readlines()

tr_len = len(tr_lines)
print ('Train_Lines', tr_len)

('Train_Lines', 800)


In [3]:
predict = []
tr_set = []


for i in range(tr_len):
    line = tr_lines[i]
    tr_line_len = len(line)
    for l in range(tr_line_len):
        if l == 1:
            predict.append(line[0:1])

for tr_l in tr_lines:       
    tr_l = tr_l.replace("0\t", "")
    tr_l = tr_l.replace("1\t", "")
    tr_l = tr_l.replace("\n", "")
    tr_l = tr_l.split()       
    tr_set.append(tr_l)

In [4]:
tr_feature = []
fr_cnt = 100001

In [5]:
for tr in tr_set:
    tr_feature_set = [0]*fr_cnt
    for v in tr:
        tr_feature_set[int(v)] = 1
    tr_feature.append(tr_feature_set)  

In [6]:
#read test file
test_data = open('C:/Users/saish/Desktop/A_PR2/test.dat','r')
ts_lines = test_data.readlines()

#Get number of records
ts_len = len(ts_lines)
print ('Test_Lines', ts_len)

('Test_Lines', 350)


In [7]:
ts_set = []
for ts_l in ts_lines:       
    ts_l = ts_l.replace("0\t", "")
    ts_l = ts_l.replace("1\t", "")
    ts_l = ts_l.replace("\n", "")
    ts_l = ts_l.split()   
    ts_set.append(ts_l)

In [8]:
ts_feature = []
fs_cnt = 100001

In [9]:
for ts in ts_set:
    ts_feature_set = [0]*fs_cnt
    for v in ts:
        ts_feature_set[int(v)] = 1
    ts_feature.append(ts_feature_set)  

# SAMPLING


In [10]:
maj_c = []
min_c = []

for cnt, val in enumerate(predict):
         if  val == '1':
             min_c.append(cnt)
         else:
             maj_c.append(cnt)

# Majority class split

In [11]:
#majority class split
maj_len = len(maj_c)
maj_70 = int((maj_len)*0.70)
maj_70_ind = []
maj_30_ind = []


#get feature set index
for i in range(maj_len):
    if (i<= maj_70):
        maj_70_ind.append(maj_c[i])
    else:
        maj_30_ind.append(maj_c[i])
        


In [12]:
#get feature set
new_tr_ft_mj = []
new_tr_pd_mj = []
for i in maj_70_ind:
    new_tr_ft_mj.append(tr_feature[i])
    new_tr_pd_mj.append(predict[i])
    

new_vl_ft_mj = []
new_vl_pd_mj = []
for i in maj_30_ind:
    new_vl_ft_mj.append(tr_feature[i])
    new_vl_pd_mj.append(predict[i])

In [13]:
print ('length of train', len(new_tr_ft_mj))
print ('length of test', len(new_vl_ft_mj))

('length of train', 506)
('length of test', 216)


# Minority class split

In [14]:
#majority class split
min_len = len(min_c)
min_70 = int((min_len)*0.70)
min_70_ind = []
min_30_ind = []


#get feature set index
for i in range(min_len):
    if (i<= min_70):
        min_70_ind.append(min_c[i])
    else:
        min_30_ind.append(min_c[i])
        


In [15]:
#get feature set
new_tr_ft_mi = []
new_tr_pd_mi = []
for i in min_70_ind:
    new_tr_ft_mi.append(tr_feature[i])
    new_tr_pd_mi.append(predict[i])
    

new_vl_ft_mi = []
new_vl_pd_mi = []
for i in min_30_ind:
    new_vl_ft_mi.append(tr_feature[i])
    new_vl_pd_mi.append(predict[i])

print ('length of train', len(new_tr_ft_mi))
print ('length of test', len(new_vl_ft_mi))

('length of train', 55)
('length of test', 23)


In [16]:
final_tr_set = new_tr_ft_mj + new_tr_ft_mi
print ('length of final train', len(final_tr_set))

final_tr_pred = new_tr_pd_mj + new_tr_pd_mi
print ('length of final train pred', len(final_tr_pred))

final_vl_set = new_vl_ft_mj + new_vl_ft_mi
print ('length of final test', len(final_vl_set))

final_vl_pred = new_vl_pd_mj + new_vl_pd_mi
print ('length of final test pred', len(final_vl_pred))


('length of final train', 561)
('length of final train pred', 561)
('length of final test', 239)
('length of final test pred', 239)


# SVD

from imblearn.over_sampling import SMOTE
tr_feature_sm, predict_sm = SMOTE().fit_sample(tr_feature, predict)
from collections import Counter print(sorted(Counter(predict_sm).items()))

In [17]:
svd = TruncatedSVD(algorithm='randomized', n_components=800, n_iter=10, random_state=42)
tr_svd = svd.fit_transform(tr_feature, predict)
final_tr_set_svd = svd.transform(final_tr_set)
final_vl_set_svd = svd.transform(final_vl_set)
ts_svd = svd.transform(ts_feature)
print(svd.explained_variance_ratio_.sum())  

1.0


In [18]:
print(tr_svd.shape)
print(final_tr_set_svd.shape)
print(final_vl_set_svd.shape)

(800L, 800L)
(561L, 800L)
(239L, 800L)


In [19]:
classifiers_list = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors=9),
    "Linear SVM": SVC(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier( ),
    "Neural Net1": MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(19, 5), random_state=1),
    #"Neural Net1": MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(800, 48), learning_rate='adaptive'),
    "Naive Bayes": GaussianNB(),
    "Adaboost": AdaBoostClassifier(n_estimators= 200),
    "ExtraTree": ExtraTreesClassifier(),
}

In [35]:
classifiers_count = len(classifiers_list.keys())
 
def classify_fn(X_train, Y_train, X_test, Y_test, verbose = True):
    df_results = pd.DataFrame(data=np.zeros(shape=(classifiers_count,5)), columns = ['classifier', 'Recall', 'F1', 'Precision', 'Accuracy'])
    count = 0
    for c_name, classifier in classifiers_list.items():
        
        classifier.fit(X_train, Y_train)
        pred = []
        pred = classifier.predict(X_test)
        
        
        cv1 = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
        scores = cross_val_score(classifier, final_tr_set, final_tr_pred, cv=cv1)
        print ('Classifier', c_name)
        print ('Cross validation', scores)
        print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
        recall = recall_score(Y_test, pred, average='weighted')
        f1 = f1_score(Y_test, pred, average='weighted')
        p = precision_score(Y_test, pred, average='weighted')
        acc =  accuracy_score(Y_test, pred)
        
        
        df_results.loc[count,'classifier'] = c_name
        df_results.loc[count,'Recall'] = recall
        df_results.loc[count,'F1'] = f1
        df_results.loc[count,'Precision'] = p
        df_results.loc[count,'Accuracy'] = acc
        
        count+=1
    return df_results

In [36]:
df_results = classify_fn(final_tr_set_svd, final_tr_pred, final_vl_set_svd, final_vl_pred)
print(df_results.sort_values(by='F1', ascending=False))

('Classifier', 'Gradient Boosting Classifier')
('Cross validation', array([ 0.89349112,  0.87573964,  0.93491124,  0.92307692,  0.91715976]))
Accuracy: 0.91 (+/- 0.04)
('Classifier', 'Adaboost')
('Cross validation', array([ 0.8816568 ,  0.88757396,  0.94674556,  0.92307692,  0.92307692]))
Accuracy: 0.91 (+/- 0.05)
('Classifier', 'Neural Net1')
('Cross validation', array([ 0.87573964,  0.8816568 ,  0.92307692,  0.88757396,  0.92899408]))
Accuracy: 0.90 (+/- 0.04)
('Classifier', 'Logistic Regression')
('Cross validation', array([ 0.87573964,  0.8816568 ,  0.92307692,  0.88757396,  0.92307692]))
Accuracy: 0.90 (+/- 0.04)
('Classifier', 'Decision Tree')
('Cross validation', array([ 0.89940828,  0.82840237,  0.9112426 ,  0.90532544,  0.90532544]))
Accuracy: 0.89 (+/- 0.06)
('Classifier', 'ExtraTree')
('Cross validation', array([ 0.90532544,  0.89349112,  0.9408284 ,  0.90532544,  0.91715976]))
Accuracy: 0.91 (+/- 0.03)


c:\users\saish\anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\saish\anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


('Classifier', 'Naive Bayes')
('Cross validation', array([ 0.87573964,  0.8816568 ,  0.91715976,  0.88757396,  0.91715976]))
Accuracy: 0.90 (+/- 0.04)
('Classifier', 'Linear SVM')
('Cross validation', array([ 0.87573964,  0.8816568 ,  0.91715976,  0.88757396,  0.91715976]))
Accuracy: 0.90 (+/- 0.04)
('Classifier', 'Nearest Neighbors')
('Cross validation', array([ 0.87573964,  0.8816568 ,  0.91715976,  0.88757396,  0.91715976]))
Accuracy: 0.90 (+/- 0.04)
('Classifier', 'Random Forest')
('Cross validation', array([ 0.8816568 ,  0.86982249,  0.92899408,  0.89349112,  0.92307692]))
Accuracy: 0.90 (+/- 0.05)
                     classifier    Recall        F1  Precision  Accuracy
0  Gradient Boosting Classifier  0.953975  0.951432   0.951245  0.953975
2                   Neural Net1  0.937238  0.935266   0.933989  0.937238
1                      Adaboost  0.941423  0.931877   0.939721  0.941423
4                 Decision Tree  0.895397  0.901444   0.909757  0.895397
3           Logistic Reg

# CLASSIFIERS

gb = GradientBoostingClassifier()
gb = gb.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = gb.predict(ts_svd)

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_gb.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

In [33]:
ab = AdaBoostClassifier(n_estimators= 100)
ab = ab.fit(final_tr_set_svd, final_tr_pred)

pred = []
pred = ab.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_AB.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

et = ExtraTreesClassifier()
et = et.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = et.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_et.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

sv = SVC()
sv = sv.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = sv.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_sv.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb = nb.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = nb.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_nb.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

rf = RandomForestClassifier()
rf = rf.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = rf.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_rf.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100,50), random_state=1)
nn = nn.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = nn.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_nn.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()


nn = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(400,2), learning_rate='adaptive')
nn = nn.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = nn.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_nn_1.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

lr = LogisticRegression()
lr = lr.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = lr.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_lr.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()

dt = tree.DecisionTreeClassifier()
dt = dt.fit(final_tr_set_svd, final_tr_pred)


pred = []
pred = dt.predict(ts_svd)
#print pred

fo=open('C:/Users/saish/Desktop/A_PR2/SVD/format_dt.txt','a+')
for i in range (len(pred)):
    #print ('i',pred[i])
    fo.write(pred[i]+'\n')
fo.close()